In [1]:
import os
import random
import numpy as np
from skimage import io
from skimage.color import rgba2rgb
from skimage.util import img_as_ubyte

# PATHS 
input_root = r'C:\Users\anand.252\OneDrive - The Ohio State University\OSU\Summer 2025\Computer Vision\Group Project\Disease Project\imgs_with_good_background'
output_root = r'C:\Users\anand.252\OneDrive - The Ohio State University\OSU\Summer 2025\Computer Vision\Group Project\Disease Project\rohit_lighting_altered'

# Lighting simulation parameters 
gen_seed = 34352
lower_frac = 0.5
upper_frac = 1.5
switch_frac_threshold = 1.0
min_ambient_frac = 0.03

sun_horz_lighting_color_vec = np.array([1.0, 0.81, 0.65]).reshape(1, 1, 3)
noon_lighting_color_vec = np.array([1.0, 0.96, 0.93]).reshape(1, 1, 3)

random.seed(gen_seed)

# Traverse and process all images 
for root, _, files in os.walk(input_root):
    for fname in files:
        if fname.lower().endswith((".jpg", ".jpeg", ".png")):
            input_path = os.path.join(root, fname)

            try:
                img = io.imread(input_path)

                if img.ndim == 2:  # Grayscale
                    print(f"Skipping grayscale image: {input_path}")
                    continue
                elif img.shape[-1] == 4:  # RGBA to RGB
                    img = rgba2rgb(img)

                # Normalize
                img_raw_normed = img.astype(np.float64) / 255.0

                # Lighting modification
                selected_intensity_frac = random.uniform(lower_frac, upper_frac)
                if selected_intensity_frac >= switch_frac_threshold:
                    res_img_raw = (
                        selected_intensity_frac * noon_lighting_color_vec * img_raw_normed
                        + min_ambient_frac * noon_lighting_color_vec
                    )
                else:
                    res_img_raw = (
                        selected_intensity_frac * sun_horz_lighting_color_vec * img_raw_normed
                    )

                # Final image conversion
                fin_procd_img = np.round(np.clip(res_img_raw, 0.0, 1.0) * 255.0).astype(np.uint8)

                # Compute relative path and save
                rel_path = os.path.relpath(input_path, input_root)
                rel_path_png = os.path.splitext(rel_path)[0] + ".png"
                output_path = os.path.join(output_root, rel_path_png)

                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                io.imsave(output_path, fin_procd_img)

            except Exception as e:
                print(f"❌ Error processing {input_path}: {e}")

print("✅ All images processed and saved with lighting variations.")

✅ All images processed and saved with lighting variations.
